In [ ]:
import schedule
#import random
import time
import pyodbc
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import win32com.client as win32
#import pyautogui
import mysql.connector
from chamada import Chamada
from datetime import datetime
#MUDANÇA NA BIBLIOTECA TIME A SER ALTERADA NA FONTE
time.clock = time.time

### CONEXÕES COM OS BANCOS NOMES ALTERADOS

In [ ]:
def con():
    import mysql.connector
    con = mysql.connector.connect(host = Chamada.nadaver('A'),
                                  database= Chamada.aver('B'),
                                 user=Chamada.ver('C'),
                                 password =Chamada.prf('D'))
    return con

def conec():
    #conexão
    server = Chamada.aver('A')
    database = Chamada.nadaver('B')
    username = Chamada.nadaver('C')
    password = Chamada.nadaver('D')
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    return cnxn
                       
def conect():
    #conexão
    server = Chamada.nadaver('F')
    database = Chamada.nadaver('G')
    username = Chamada.nadaver('H') 
    password = Chamada.nadaver('I') 
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    return cnxn

### E-MAIL AUTOMÁTICO

In [ ]:
def resultados_Base_Lotofacil():
    sql_server = conect()
    sql_cursor = sql_server.cursor()
    leitura=pd.read_sql('''SELECT top 5 * FROM HISTORICO_BASE order by ID_RESULTADO desc ''', sql_server)
    sql_server.commit()
    sql_cursor.close()
    leitura = leitura.rename(columns={'ID_RESULTADO':'CONCURSO',
               'NUMERO13':'13_PONTOS','NUMERO14':'14_PONTOS','NUMERO15':'15_PONTOS'})
    leitura=leitura[['CONCURSO','13_PONTOS','14_PONTOS','15_PONTOS']]
    return leitura  
    
def E_mail_bom_facil(concurso,sorteado,treze,quatorze,quinze,trevo):
    outlook = win32.Dispatch('outlook.application')
    email = outlook.CreateItem(0)
    email.To= Chamada.aver("FULANO@gmail.com")
    stats = resultados_Base_Lotofacil()
    #para enviar para mais de um destinatário basta colocar ; e escrever o email na sequencia
    email.Subject = "E-mail automático de segurança"
    email.HTMLBody = '''
    <p> LOTOFACIL</p>
    <p> A atualização do servidor foi concluida!!!</p>
    <p> Número do concurso:{concurso} </p>
    <p> Números do sorteio: {sorteado}  </p>
    <p></p>
    <p>====================RESULTADOS=======================</p>
    {stats}
    <p>=====================================================</p>
    <p></p>
    <p>=====Quantidade de ganhadores====== </p>
    <p> 13 acertos: {treze} </p>
    <p> 14 acertos: {quatorze} </p>
    <p> 15 acertos: {quinze} </p>
    <p> {trevo}{trevo}{trevo} </p>
    '''.format(concurso = concurso,sorteado = sorteado,
               treze=treze,quatorze=quatorze,quinze=quinze,trevo = trevo
               ,stats=stats.to_html())
    #anexo = caminho do documento  + extensão: 
    #C:\Users\Weskley\Documents\arquivo.xlsx
    #email.attachments.add(anexo)
    email.Send()

### COLETA DOS DADOS DO BANCO SQL SERVER PARA ENVIO

In [ ]:
def pega_base_facil():
    cnxn =conec()

    sql_server = cnxn.cursor()
    leitura=pd.read_sql('''select ID_RESULTADO,NUMERO1,NUMERO2,NUMERO3 ,NUMERO4 ,NUMERO5 
    ,NUMERO6 ,NUMERO7 ,NUMERO8 ,NUMERO9 ,NUMERO10 ,NUMERO11 ,NUMERO12 ,NUMERO13
    ,NUMERO14 ,NUMERO15  from LOTOFACIL''', cnxn)
    cnxn.commit()
    sql_server.close()
    conversao = leitura.to_dict()
    conversao = list(conversao.values())
    return conversao


def historico_facil():
    cnxn = conect()
    sql_server = cnxn.cursor()
    leitura=pd.read_sql('''DECLARE @ID INT
        SELECT @ID = MAX(ID_RESULTADO) FROM HISTORICO_BASE
        SELECT * FROM HISTORICO_BASE WHERE ID_RESULTADO = @ID''', cnxn)
    cnxn.commit()
    sql_server.close()
    conversao = leitura.to_dict()
    conversao = list(conversao.values())
    return conversao

def ultimo_resultado():
    sql_server = conect()
    sql_cursor = sql_server.cursor()
    leitura=pd.read_sql('''SELECT top 1 ID_RESULTADO,F1,
        F2,F3,F4,F5,F6,F7
        ,F8,F9,F10,F11,F12,F13,
        F14,F15  FROM RESULTADOS_FACIL$ 
    order by ID_RESULTADO desc''', sql_server)
    sql_server.commit()
    sql_cursor.close()
    conversao = leitura.to_dict()
    conversao = list(conversao.values())
    
    mysql = con()
    mysql_cursor = mysql.cursor()
    insercao = ''
    insercao = f'''INSERT INTO RESULTADOS_LOTOFACIL (ID_RESULTADO,NUMERO1,
    NUMERO2,NUMERO3,NUMERO4,NUMERO5,NUMERO6,NUMERO7
    ,NUMERO8,NUMERO9,NUMERO10,NUMERO11,NUMERO12,NUMERO13,
    NUMERO14,NUMERO15) VALUES(
    {conversao[0][0]},{conversao[1][0]},{conversao[2][0]},
    {conversao[3][0]},{conversao[4][0]},{conversao[5][0]},
    {conversao[6][0]},{conversao[7][0]},{conversao[8][0]},
    {conversao[9][0]},{conversao[10][0]},{conversao[11][0]},
    {conversao[12][0]},{conversao[13][0]},{conversao[14][0]},
    {conversao[15][0]})'''
    #comando execute
    mysql_cursor.execute(insercao) 
    mysql.commit()
    mysql_cursor.close()

### ENVIO BÁSICO PARA BASE MYSQL

In [ ]:
def insere_base_facil():
    conversao = pega_base_facil()
    
    cone = con()
    
    conexao = cone.cursor()
    conexao.execute('TRUNCATE LOTOFACIL_100MIL')    
    a = 0
    while a <100000:
        insercao = ''
        insercao = '''INSERT INTO LOTOFACIL_100MIL (ID_RESULTADO,NUMERO1,NUMERO2,NUMERO3 ,NUMERO4 ,NUMERO5 
        ,NUMERO6 ,NUMERO7 ,NUMERO8 ,NUMERO9 ,NUMERO10 ,NUMERO11 ,NUMERO12 ,NUMERO13
        ,NUMERO14 ,NUMERO15) VALUES '''
        insercao = insercao + f'''({conversao[0][a]},
                {conversao[1][a]},{conversao[2][a]},{conversao[3][a]},{conversao[4][a]},
                {conversao[5][a]},{conversao[6][a]},{conversao[7][a]},{conversao[8][a]},
                {conversao[9][a]},{conversao[10][a]},{conversao[11][a]},{conversao[12][a]},
                {conversao[13][a]},{conversao[14][a]},{conversao[15][a]})'''
        a = a + 1
        for i in range(999):
            insercao = insercao + ','+f'''({conversao[0][a]},
                {conversao[1][a]},{conversao[2][a]},{conversao[3][a]},{conversao[4][a]},
                {conversao[5][a]},{conversao[6][a]},{conversao[7][a]},{conversao[8][a]},
                {conversao[9][a]},{conversao[10][a]},{conversao[11][a]},{conversao[12][a]},
                {conversao[13][a]},{conversao[14][a]},{conversao[15][a]})'''
            a = a + 1
        #comando execute
        time.sleep(1)
        conexao.execute(insercao) 
    cone.commit()
    conexao.close()



def insere_historico_facil():
    conversao = historico_facil()
    
    cone= con()
    
    conexao = cone.cursor()   
    insercao = ''
    insercao = '''INSERT INTO HISTORICO_BASE_LOTOFACIL (ID_RESULTADO
    ,PONTOS_13,PONTOS_14,PONTOS_15 ) VALUES '''
    insercao = insercao + f'''({conversao[3][0]},
            {conversao[0][0]},{conversao[1][0]},{conversao[2][0]})'''
    #comando execute
    time.sleep(1)
    conexao.execute(insercao) 
    cone.commit()
    conexao.close()

### CRIAÇÃO DE HISTÓRICO DIÁRIO DA BASE ENVIADA

In [ ]:
def dados_por_concurso_facil(numero):
    numero = numero
    concurso_novo = 'LOTOFACIL_' + str(numero)
    cnxn = conect()
    #coleta do número do sorteio
    cursor = cnxn.cursor()
    cursor.execute(f'''CREATE DATABASE {concurso_novo}''')
    cnxn.commit()
    cursor.close()
    cursor = cnxn.cursor()
    cursor.execute(f'use {concurso_novo}')
    cursor.execute('''CREATE TABLE LOTOFACIL_100MIL(
    ID_RESULTADO INT NOT NULL,
    NUMERO1 TINYINT NOT NULL,
    NUMERO2 TINYINT NOT NULL,
    NUMERO3 TINYINT NOT NULL,
    NUMERO4 TINYINT NOT NULL,
    NUMERO5 TINYINT NOT NULL,
    NUMERO6 TINYINT NOT NULL,
    NUMERO7 TINYINT NOT NULL,
    NUMERO8 TINYINT NOT NULL,
    NUMERO9 TINYINT NOT NULL,
    NUMERO10 TINYINT NOT NULL,
    NUMERO11 TINYINT NOT NULL,
    NUMERO12 TINYINT NOT NULL,
    NUMERO13 TINYINT NOT NULL,
    NUMERO14 TINYINT NOT NULL,
    NUMERO15 TINYINT NOT NULL )''')
    cursor.execute(f''' INSERT INTO {concurso_novo}.dbo.LOTOFACIL_100MIL
    SELECT * FROM LOTOFACIL_100MIL.dbo.LOTOFACIL''')
    cnxn.commit()
    cursor.close()

### CÓDIGOS DE RECUPERAÇÃO PARA A BASE ONLINE

In [ ]:
def ultimo_resultado_MASSA():
    sql_server = conect()
    sql_cursor = sql_server.cursor()
    leitura=pd.read_sql('''SELECT  TOP 46 ID_RESULTADO,F1,
        F2,F3,F4,F5,F6,F7
        ,F8,F9,F10,F11,F12,F13,
        F14,F15 FROM RESULTADOS_FACIL$ 
    order by ID_RESULTADO desc ''', sql_server)
    sql_server.commit()
    sql_cursor.close()
    conversao = leitura.to_dict()
    conversao = list(conversao.values())
    
    mysql = con()
    mysql_cursor = mysql.cursor()
    #mysql_cursor.execute('TRUNCATE RESULTADOS_LOTOFACIL') 
    a = 0
    while a <46:#VARIA
        insercao = ''
        insercao = f'''INSERT INTO RESULTADOS_LOTOFACIL (ID_RESULTADO,NUMERO1,
        NUMERO2,NUMERO3,NUMERO4,NUMERO5,NUMERO6,NUMERO7
        ,NUMERO8,NUMERO9,NUMERO10,NUMERO11,NUMERO12,NUMERO13,
        NUMERO14,NUMERO15) VALUES(
        {conversao[0][a]},
        {conversao[1][a]},{conversao[2][a]},{conversao[3][a]},
        {conversao[4][a]},{conversao[5][a]},{conversao[6][a]},
        {conversao[7][a]},{conversao[8][a]},{conversao[9][a]},
        {conversao[10][a]},{conversao[11][a]},{conversao[12][a]},
        {conversao[13][a]},{conversao[14][a]},{conversao[15][a]})'''
        a = a + 1
        print(a)
        for i in range(45):#VARIA
            insercao = insercao + ','+f'''({conversao[0][a]},
        {conversao[1][a]},{conversao[2][a]},{conversao[3][a]},
        {conversao[4][a]},{conversao[5][a]},{conversao[6][a]},
        {conversao[7][a]},{conversao[8][a]},{conversao[9][a]},
        {conversao[10][a]},{conversao[11][a]},{conversao[12][a]},
        {conversao[13][a]},{conversao[14][a]},{conversao[15][a]})'''
            a = a + 1
        #comando execute
        time.sleep(1)
        mysql_cursor.execute(insercao)
    mysql.commit()
    mysql_cursor.close()
    
def historico_facil_MASSA():
    cnxn = conect()
    sql_server = cnxn.cursor()
    leitura=pd.read_sql('''SELECT  * FROM HISTORICO_BASE ORDER BY ID_RESULTADO  ''', cnxn)
    cnxn.commit()
    sql_server.close()
    conversao = leitura.to_dict()
    conversao = list(conversao.values())
    return conversao

def insere_historico_facil_MASSA():
    conversao = historico_facil_MASSA()
    
    cone= con()
    
    conexao = cone.cursor()
    conexao.execute('TRUNCATE HISTORICO_BASE_LOTOFACIL') 
    a = 0
    while a <4:
        insercao = ''
        insercao = '''INSERT INTO HISTORICO_BASE_LOTOFACIL (ID_RESULTADO
        ,PONTOS_13,PONTOS_14,PONTOS_15 ) VALUES '''
        insercao = insercao + f'''({conversao[3][a]},
                {conversao[0][a]},{conversao[1][a]},{conversao[2][a]})'''
        a = a + 1
        print(a)
        for i in range(3):
            insercao = insercao + ','+f'''({conversao[3][a]},
                {conversao[0][a]},{conversao[1][a]},{conversao[2][a]})'''
            a = a + 1
        #comando execute
        time.sleep(1)
        conexao.execute(insercao) 
    cone.commit()
    conexao.close()

## FUNÇÃO FINAL

AQUI TEMOS A JUNÇÃO DE CONEXÃO DE 2 BASES DE DADOS DE DISTINTAS LINGUAGENS
, AUTOMAÇÃO WEB COM SELENIUM , ENVIO DE RELATÓRIO POR E-MAIL E UM PROCESSO AUTOMÁTICO DE VERIFICAÇÃO DIÁRIA E ATUALIZAÇÃO CONFORME OS SORTEIOS DA LOTERIA

In [ ]:
def Lotofacil():
    #conexão
    trevo = '\U0001f340'
    cnxn = conect()
    #coleta do número do sorteio
    cursor = cnxn.cursor()
    sorteio=pd.read_sql('''DECLARE @MAXIMO INT
    SELECT @MAXIMO = MAX(ID_CONCURSO) FROM SORTEIO
    SELECT NUMERO_CONCURSO FROM SORTEIO WHERE ID_CONCURSO = @MAXIMO''', cnxn)
    cnxn.commit()
    cursor.close()
    
    #conversao
    conversao = sorteio.to_dict()
    conversao = list(conversao.values())
    numero_sorteio = conversao[0][0]
    
    #argumento de ocultação
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    
    # driver de funcionamento do navegador
    driver = webdriver.Chrome(chrome_options = chrome_options)
    
    # nome do site
    driver.get("""https://www.google.com/search?q=lotofacil&oq=lotofa&aqs=chrome.0.69i59j69i57j69i59j0i131i395i433l2j69i60l3.2792j1j7&sourceid=chrome&ie=UTF-8 """)

    # print(driver.title)
    sorteado = [] 
    
    #coleta do concurso atual do site /html/body/div[7]/div/div[9]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/span/div/div[1]/div[1]/div[1]/span[3]/span
    concurso = driver.find_element_by_class_name('qLLird').text
    concurso_novo = concurso[9:13]
    print('LOTOFACIL')
    print(concurso)
    print(numero_sorteio,concurso_novo)
    numero_sorteio = str(numero_sorteio)
    #verificação se vai ou não atualizar
    if numero_sorteio <concurso_novo:
        concurso = (concurso[0:8] + "_" + concurso[9:13] + "_" + concurso[15:17] + '_' + concurso[18:20] + '_' + concurso[21:23])
        b = driver.find_element_by_class_name('Z30kQd').text
        numero = b[0:2]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[2:4]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[4:6]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[6:8]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[8:10]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[11:13]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[13:15]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[15:17]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[17:19]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[19:21]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[22:24]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[24:26]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[26:28]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[28:30]
        numero = int(numero)
        sorteado.append(numero)
        numero = b[30:32]
        numero = int(numero)
        sorteado.append(numero)
        
        #bloco de inserção do numero do sorteio
        cursor = cnxn.cursor()
        cursor.execute(f'''INSERT INTO SORTEIO 
        (NUMERO_CONCURSO) values ({concurso_novo})''')
        time.sleep(10)
        cnxn.commit()
        cursor.close()
        
        #faz parte da criação do banco com os dados de cada concurso
        #dos clientes onde precisam ter os 100mil jogos(OK)
        #e os clientes que compraram cada um dos jogos(falta fazer)
        #dados_por_concurso_facil(concurso_novo)
        
        #inserindo o resultado
        cursor = cnxn.cursor()
        cursor.execute(f'''INSERT INTO RESULTADOS_FACIL$ (F1,F2,F3,F4,F5,
        F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,ID_RESULTADO) values ({sorteado[0]},{sorteado[1]},{sorteado[2]},
        {sorteado[3]},{sorteado[4]},{sorteado[5]},{sorteado[6]},{sorteado[7]},{sorteado[8]},
        {sorteado[9]},{sorteado[10]},{sorteado[11]},{sorteado[12]},{sorteado[13]},{sorteado[14]}
        ,{concurso_novo})''')
        time.sleep(10)
        cnxn.commit()
        cursor.close()
        
        #atualiza tudo
        cursor = cnxn.cursor()
        cursor.execute("SET NOCOUNT ON; exec ATUALIZACAO_SEMANAL ")
        cnxn.commit()
        cursor.close()
        
        
        #alimentação da base de 100mil numeros
        insere_base_facil()
        insere_base_facil_vip() 

        #insere os historicos
        insere_historico_facil()
        insere_historico_facil_vip()
        
        #insere o ultimo concurso
        ultimo_resultado()
        
        #coleta dos jogos dos assinantes(numeros)
        cursor = cnxn.cursor()
        ganhadores=pd.read_sql(f'select NUMERO13,NUMERO14,NUMERO15 from QT_APOSTAS_GANHADORAS',cnxn)
        time.sleep(10)
        cnxn.commit()
        cursor.close()
        
        #gera a base com os jogos daquele consurso
        #SOMENTE RODAR QUANDO FOR OFICIAL E ESTIVER COM TRANFERÊNCIA DE 
        # JOGOS DE CLIENTES E A QUANTIDADE DE GANHADORES 
        #dados_por_concurso_facil(concurso_novo)
        
        #conversão dos dados
        inversao = ganhadores.to_dict()
        inversao = list(inversao.values())
        treze = inversao[0][0]
        quatorze = inversao[1][0]
        quinze = inversao[2][0]
        
        #e-mail de segurança quando atualizar
        E_mail_bom_facil(concurso,sorteado,treze,quatorze,quinze,trevo)
        E_mail_bom_facil_vip(concurso,sorteado,treze,quatorze,quinze,trevo)

In [ ]:
def roda():
    from datetime import date
    # datetime object containing current date and time
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    trevo = '\U0001f340'
    print("date and time =", dt_string)
    Lotofacil()
    E_mail_ciclo(trevo)
    # datetime object containing current date and time
    now = datetime.now()

    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("date and time =", dt_string)

In [ ]:
schedule.every().monday.at("12:00").do(roda)
schedule.every().tuesday.at("12:00").do(roda)
schedule.every().wednesday.at("12:00").do(roda)
schedule.every().thursday.at("12:00").do(roda) 
schedule.every().friday.at("12:00").do(roda)
schedule.every().saturday.at("12:00").do(roda) 
print("=====================#=====================#=======================")
schedule.every().monday.at("13:00").do(roda)
schedule.every().tuesday.at("13:00").do(roda)
schedule.every().wednesday.at("13:00").do(roda)
schedule.every().thursday.at("13:00").do(roda)
schedule.every().friday.at("13:00").do(roda)
schedule.every().saturday.at("13:00").do(roda)
while 1:
    schedule.run_pending()
    time.sleep(1) 